- This one is based on the great starting [notebook](https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-576) by @NICHOLAS BROAD.
- please note: I didn't use the deberta-base model I choose here to finetune, this is just a demo.
- In the postprocessing part of inference, I changed **tokens** to **token_map** in two lines. This will bring a huge improvement. There is the [inference notebook](https://www.kaggle.com/code/takanashihumbert/piidd-deberta-model-starter-inference)

In [1]:
!nvidia-smi

Sat Mar 30 05:34:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import runtime

In [4]:
class CFG:
  exp='exp044_full_train'
  max_len = 512
  stride = 128
  model_name = 'microsoft/deberta-v3-large'
  seed=42
  target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM',
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM',
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
  ]

In [5]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = f'/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/{CFG.exp}/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [6]:
!pip install seqeval evaluate -q
!pip install sentencepiece

!pip install transformers==4.31.0
os.system('pip install -q tokenizers')
!pip install tokenizers==0.13.3

!pip install accelerate -U



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Suc

In [7]:
import json
import argparse
from itertools import chain
from functools import partial

import torch
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features, DatasetDict
# import datasets
import numpy as np

In [8]:
import pandas as pd

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
import json
input_path = '/content/drive/MyDrive/Kaggle/competition/PIIDD/input/'
data = json.load(open(input_path+"train.json"))

print(len(data))
print(data[0].keys())

6807
dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])


In [11]:
# input_path = '/content/drive/MyDrive/Kaggle/competition/PIIDD/input/'
# external_data = pd.read_csv(input_path+'pii_dataset.csv')

# external_data=external_data[['text','document','tokens','trailing_whitespace','labels']].rename(columns={'text':'full_text','labels':'provided_labels'})

# ds_external = Dataset.from_pandas(external_data)


In [12]:
import json
input_path = '/content/drive/MyDrive/Kaggle/competition/PIIDD/input/'
external_data = json.load(open(input_path+"mixtral-8x7b-v1.json"))

print(len(external_data))
print(external_data[0].keys())

2355
dict_keys(['document', 'full_text', 'tokens', 'labels', 'trailing_whitespace'])


In [13]:
train_data=data+external_data

for i in range(len(train_data)):
  train_data[i]['document']=i+1

In [14]:
for i in range(len(train_data)):
  for idx,labels in enumerate(train_data[i]['labels']):
    if labels in CFG.target:
      train_data[i]['labels'][idx]=train_data[i]['labels'][idx][2:]

In [15]:
all_labels = [
    'EMAIL', 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM',
    'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME','O'
  ]
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

print(id2label)

{0: 'EMAIL', 1: 'ID_NUM', 2: 'NAME_STUDENT', 3: 'PHONE_NUM', 4: 'STREET_ADDRESS', 5: 'URL_PERSONAL', 6: 'USERNAME', 7: 'O'}


In [16]:
target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM',
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM',
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

In [17]:
def is_nested_list(lst):
    return any(isinstance(item, list) for item in lst)

In [18]:
def tokenize(example, tokenizer, label2id):
    text = []

    # these are at the character level
    labels = []
    targets = []

    for t, l, ws in zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"]):
        text.append(t)
        #文字レベルでlabelを振っている．
        labels.extend([l]*len(t))

        if l in CFG.target:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")

    target_num = sum(targets)
    labels = np.array(labels)

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=CFG.max_len, stride=CFG.stride,return_overflowing_tokens=True)
    text = "".join(text)

    token_samples=[]

    if not is_nested_list(tokenized.offset_mapping):
      token_labels = []
      for start_idx, end_idx in tokenized.offset_mapping:

          # CLS token
          if start_idx == 0 and end_idx == 0:
              token_labels.append(label2id["O"])
              continue

          # case when token starts with whitespace
          if text[start_idx].isspace() and text[start_idx]!='\n' and text[start_idx]!='\n\n':
              start_idx += 1

          token_labels.append(label2id[labels[start_idx]])

      length = len(tokenized.input_ids)

      return [
          tokenized.input_ids,
          tokenized.token_type_ids,
          tokenized.attention_mask,
          tokenized.offset_mapping,
          tokenized.overflow_to_sample_mapping,
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
      ]

    else:
      for i in range(len(tokenized.offset_mapping)):
        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping[i]:

            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(label2id["O"])
                continue

            # case when token start
            if text[start_idx].isspace() and text[start_idx]!='\n' and text[start_idx]!='\n\n':
                start_idx += 1

            token_labels.append(label2id[labels[start_idx]])

        length = len(tokenized.input_ids[i])

        token_sample= [
          tokenized.input_ids[i],
          tokenized.token_type_ids[i],
          tokenized.attention_mask[i],
          tokenized.offset_mapping[i],
          tokenized.overflow_to_sample_mapping[i],
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
      ]
        token_samples.append(token_sample)

      return token_samples

In [19]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
tokenizer.add_tokens(['\n','\n\n'], special_tokens=True)

train_ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in train_data],
    "document": [x["document"] for x in train_data],
    "tokens": [x["tokens"] for x in train_data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in train_data],
    "provided_labels": [x["labels"] for x in train_data],
})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
%%time
data_list=[]
for example in train_ds:
    # 各サンプルに対してtokenize関数を呼び出し
    dataset_list=tokenize(example, tokenizer=tokenizer, label2id=label2id)
    for d in dataset_list:
      data_list.append(d)


train_ds=Dataset.from_dict({
  'input_ids': [x[0] for x in data_list],
  'token_type_ids': [x[1] for x in data_list],
  "attention_mask":[x[2] for x in data_list],
  'offset_mapping': [x[3] for x in data_list],
  'overflow_to_sample_mapping':[x[4] for x in data_list],
  "labels": [x[5] for x in data_list],
  "length": [x[6] for x in data_list],
  "target_num": [x[7] for x in data_list],
  "group": [x[8] for x in data_list],
  'document':[x[9] for x in data_list],
  })

# ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id}, num_proc=2)

train_ds = train_ds.class_encode_column("group")


Stringifying the column:   0%|          | 0/19846 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/19846 [00:00<?, ? examples/s]

CPU times: user 1min 13s, sys: 3.91 s, total: 1min 17s
Wall time: 1min 16s


In [21]:
for i in range(32,10000):
  if train_ds[i]['group']>0:
    break

In [22]:
from transformers import DebertaV2PreTrainedModel, DebertaV2Model
from transformers.modeling_outputs import TokenClassifierOutput
import torch.nn as nn
class LSTMHead(nn.Module):
    def __init__(self, in_features, hidden_dim, n_layers):
        super().__init__()
        self.lstm = nn.LSTM(in_features,
                            hidden_dim,
                            n_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=0.1)
        self.out_features = hidden_dim

    def forward(self, x):
        self.lstm.flatten_parameters()
        hidden, (_, _) = self.lstm(x)
        out = hidden
        return out


# Copied from transformers.models.deberta.modeling_deberta.DebertaForTokenClassification with Deberta->DebertaV2
class CustomModel(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.deberta = DebertaV2Model(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.head = LSTMHead(in_features=config.hidden_size, hidden_dim=config.hidden_size//2, n_layers=1)
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):

        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.head(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Assuming class_weights is a Tensor of weights for each class
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels
        labels = inputs.pop("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        # Reshape for loss calculation
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        if self.label_smoother is not None and "labels" in inputs:
            loss = self.label_smoother(outputs, inputs)
        else:
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

## competition metrics

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    CFG.model_name,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# FREEZE_EMBEDDINGS = False
# FREEZE_LAYERS = 6

# if FREEZE_EMBEDDINGS:
#     print('Freezing embeddings.')
#     for param in model.deberta.embeddings.parameters():
#         param.requires_grad = False

# if FREEZE_LAYERS>0:
#     print(f'Freezing {FREEZE_LAYERS} layers.')
#     for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
#         for param in layer.parameters():
#             param.requires_grad = False

In [25]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Assuming class_weights is a Tensor of weights for each class
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels
        labels = inputs.pop("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        # Reshape for loss calculation
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        if self.label_smoother is not None and "labels" in inputs:
            loss = self.label_smoother(outputs, inputs)
        else:
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

## training

In [26]:

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    report_to="none",
    evaluation_strategy="no",
    do_eval=False,
    save_total_limit=1,
    logging_steps=100,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    weight_decay=0.01
)
class_weights=torch.tensor([5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0 ,1.0]).to(device)

# Initialize Trainer with custom class weights
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    data_collator=collator,
    tokenizer=tokenizer,
    class_weights=class_weights
)


In [27]:
len(train_ds)

19846

In [28]:
%%time
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,0.832000
200,0.015900
300,0.005700
400,0.004800
500,0.005700
600,0.004300
700,0.002500
800,0.004800
900,0.003500
1000,0.001800


CPU times: user 26min 18s, sys: 9min 21s, total: 35min 40s
Wall time: 36min 49s


TrainOutput(global_step=3723, training_loss=0.025090833995796456, metrics={'train_runtime': 2208.7445, 'train_samples_per_second': 26.956, 'train_steps_per_second': 1.686, 'total_flos': 5.529485631755059e+16, 'train_loss': 0.025090833995796456, 'epoch': 3.0})

In [29]:
trainer.save_model(OUTPUT_DIR)
torch.cuda.empty_cache()

In [30]:
runtime.unassign()